# 데이터 로드

In [1]:
import pandas as pd

###  INPUT ###
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')
input_data.shape

(84840, 50)

# 데이터 전처리

In [2]:
import numpy as np

# ... (Data preprocessing code here)
df = input_data.drop(columns=['frmDist'])
df = df.sort_values(by='date') # 날짜 오름차순 정렬

In [3]:
df.columns

Index(['date', 'inTp', 'inHd', 'otmsuplyqy', 'acSlrdQy', 'cunt', 'ph', 'outTp',
       'outWs', 'daysuplyqy', 'inCo2', 'ec', 'frmYear', 'frmWeek',
       'frtstGrupp', 'lefstalklt', 'frtstSetCo', 'pllnLt', 'flanGrupp',
       'frtstCo', 'flanJnt', 'tcdmt', 'frmhsFclu', 'hvstGrupp', 'hvstJnt',
       'grwtLt', 'fcluHg', 'lefLt', 'flwrCo', 'hvstCo', 'lefCunt', 'frtstJnt',
       'lefBt', 'stemThck', 'frmAr', 'frmDov', 'outtrn', 'outtrn_cumsum',
       'WaterUsage', 'WaterCost', 'FertilizerUsage', 'FertilizerCost',
       'HeatingEnergyUsage', 'HeatingEnergyCost', 'CO2Usage', 'CO2Cost',
       'MistUsageTime', 'Mist Cost', 'HeatingEnergyUsage_cumsum'],
      dtype='object')

In [4]:
# 모든 행의 값이 0인 컬럼 삭제
zero_columns = df.columns[(df == 0).all()]
zero_columns

Index(['daysuplyqy', 'lefstalklt', 'frtstSetCo', 'pllnLt', 'flanJnt',
       'hvstJnt', 'flwrCo', 'frtstJnt'],
      dtype='object')

In [5]:
df.drop(zero_columns, axis=1, inplace=True)

# NaN 포함한 행 삭제
df = df.dropna()

In [6]:
con = df[(df['date'] >= 20170101) & (df['date'] <= 20191231)]
cost = df[(df['date'] >= 20220101) & (df['date'] <= 20221231)]

In [7]:
zero_con = con.columns[(con == 0).all()]
print(zero_con)
con.drop(zero_con, axis=1, inplace=True)

zero_cost = cost.columns[(cost == 0).all()]
print(zero_cost)
cost.drop(zero_cost, axis=1, inplace=True)

Index(['tcdmt', 'WaterUsage', 'WaterCost', 'FertilizerUsage', 'FertilizerCost',
       'HeatingEnergyUsage', 'HeatingEnergyCost', 'CO2Usage', 'CO2Cost',
       'MistUsageTime', 'Mist Cost', 'HeatingEnergyUsage_cumsum'],
      dtype='object')
Index(['inTp', 'inHd', 'otmsuplyqy', 'acSlrdQy', 'cunt', 'outTp', 'outWs',
       'inCo2', 'frmYear', 'frmWeek', 'frtstGrupp', 'flanGrupp', 'frmhsFclu',
       'hvstGrupp', 'hvstCo', 'stemThck'],
      dtype='object')


<ipython-input-7-0433c8fa2dc7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con.drop(zero_con, axis=1, inplace=True)
<ipython-input-7-0433c8fa2dc7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost.drop(zero_cost, axis=1, inplace=True)


In [8]:
con.columns

Index(['date', 'inTp', 'inHd', 'otmsuplyqy', 'acSlrdQy', 'cunt', 'ph', 'outTp',
       'outWs', 'inCo2', 'ec', 'frmYear', 'frmWeek', 'frtstGrupp', 'flanGrupp',
       'frtstCo', 'frmhsFclu', 'hvstGrupp', 'grwtLt', 'fcluHg', 'lefLt',
       'hvstCo', 'lefCunt', 'lefBt', 'stemThck', 'frmAr', 'frmDov', 'outtrn',
       'outtrn_cumsum'],
      dtype='object')

In [9]:
cost.columns

Index(['date', 'ph', 'ec', 'frtstCo', 'tcdmt', 'grwtLt', 'fcluHg', 'lefLt',
       'lefCunt', 'lefBt', 'frmAr', 'frmDov', 'outtrn', 'outtrn_cumsum',
       'WaterUsage', 'WaterCost', 'FertilizerUsage', 'FertilizerCost',
       'HeatingEnergyUsage', 'HeatingEnergyCost', 'CO2Usage', 'CO2Cost',
       'MistUsageTime', 'Mist Cost', 'HeatingEnergyUsage_cumsum'],
      dtype='object')

In [10]:
import pandas as pd

# 날짜 변환
con['date'] = pd.to_datetime(con['date'], format='%Y%m%d')

# 파생 변수 생성
con['temp_difference'] = con['inTp'] - con['outTp']
con['sunlight_per_watering'] = con.apply(lambda row: row['acSlrdQy'] / row['cunt'] if row['cunt'] != 0 else 0, axis=1)
con['7day_avg_inTp'] = con['inTp'].rolling(window=7).mean()
con['temp_humidity_interaction'] = con['inTp'] * con['inHd']
con['month'] = con['date'].dt.month
con['day'] = con['date'].dt.day
con['season'] = con['month'].apply(lambda x: 'spring' if 3 <= x < 6 else 'summer' if 6 <= x < 9 else 'fall' if 9 <= x < 12 else 'winter')
con['7day_avg_inHd'] = con['inHd'].rolling(window=7).mean()

# NaN 값 처리
con.dropna(subset=['7day_avg_inTp', '7day_avg_inHd'], inplace=True)

# 원-핫 인코딩
con = pd.concat([con, pd.get_dummies(con['season'], prefix='season')], axis=1)
con.drop('season', axis=1, inplace=True)

# 'inTp'와 'outTp'의 차이를 이용한 일간 온도 변동
con['daily_temp_change'] = con['inTp'].diff()

# 'inHd'의 일간 변동
con['daily_humidity_change'] = con['inHd'].diff()

# 일출 전 후로 'acSlrdQy'가 어떻게 변하는지 체크하기 위한 지연 변수 (하루 지연)
con['acSlrdQy_lag1'] = con['acSlrdQy'].shift(1)

# 'otmsuplyqy'와 'acSlrdQy'의 비율
con['sunlight_to_otmsuplyqy_ratio'] = con.apply(lambda row: row['acSlrdQy'] / row['otmsuplyqy'] if row['otmsuplyqy'] != 0 else 0, axis=1)

# NaN 값 처리 (새로 생성한 지연 변수에 대해 NaN 값을 처리)
con.dropna(subset=['daily_temp_change', 'daily_humidity_change', 'acSlrdQy_lag1'], inplace=True)

<ipython-input-10-72fb28369dc9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con['date'] = pd.to_datetime(con['date'], format='%Y%m%d')
<ipython-input-10-72fb28369dc9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con['temp_difference'] = con['inTp'] - con['outTp']
<ipython-input-10-72fb28369dc9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [11]:
con.columns

Index(['date', 'inTp', 'inHd', 'otmsuplyqy', 'acSlrdQy', 'cunt', 'ph', 'outTp',
       'outWs', 'inCo2', 'ec', 'frmYear', 'frmWeek', 'frtstGrupp', 'flanGrupp',
       'frtstCo', 'frmhsFclu', 'hvstGrupp', 'grwtLt', 'fcluHg', 'lefLt',
       'hvstCo', 'lefCunt', 'lefBt', 'stemThck', 'frmAr', 'frmDov', 'outtrn',
       'outtrn_cumsum', 'temp_difference', 'sunlight_per_watering',
       '7day_avg_inTp', 'temp_humidity_interaction', 'month', 'day',
       '7day_avg_inHd', 'season_fall', 'season_spring', 'season_summer',
       'season_winter', 'daily_temp_change', 'daily_humidity_change',
       'acSlrdQy_lag1', 'sunlight_to_otmsuplyqy_ratio'],
      dtype='object')

In [12]:
nan_rows = df[df.isnull().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [date, inTp, inHd, otmsuplyqy, acSlrdQy, cunt, ph, outTp, outWs, inCo2, ec, frmYear, frmWeek, frtstGrupp, flanGrupp, frtstCo, tcdmt, frmhsFclu, hvstGrupp, grwtLt, fcluHg, lefLt, hvstCo, lefCunt, lefBt, stemThck, frmAr, frmDov, outtrn, outtrn_cumsum, WaterUsage, WaterCost, FertilizerUsage, FertilizerCost, HeatingEnergyUsage, HeatingEnergyCost, CO2Usage, CO2Cost, MistUsageTime, Mist Cost, HeatingEnergyUsage_cumsum]
Index: []

[0 rows x 41 columns]


In [13]:
def calculate_ratio(df, numerator, denominator, new_column_name):
    df[new_column_name] = np.where(df[denominator] != 0, df[numerator] / df[denominator], 0)
    return df

def calculate_rolling_average(df, column, window_size=7):
    return df[column].rolling(window=window_size).mean().fillna(method='ffill')

def calculate_rolling_std(df, column, window_size=7):
    return df[column].rolling(window=window_size).std()

def calculate_recent_trend(df, column, window_size=7):
    return df[column] - df[column].rolling(window=window_size).mean()

cost = calculate_ratio(cost, 'CO2Cost', 'CO2Usage', 'CO2_Usage_Cost_Ratio')
cost = calculate_ratio(cost, 'Mist Cost', 'MistUsageTime', 'Mist_Time_Cost_Ratio')

# 통계적 집계
cost['7day_avg_WaterUsage'] = calculate_rolling_average(cost, 'WaterUsage')
cost['7day_avg_FertilizerUsage'] = calculate_rolling_average(cost, 'FertilizerUsage')
cost['7day_avg_CO2Usage'] = calculate_rolling_average(cost, 'CO2Usage')

# 다항/상호작용 파생 변수 생성
cost['ph_ec_interaction'] = cost['ph'] * cost['ec']

# 날짜 처리
cost['date'] = pd.to_datetime(cost['date'], format='%Y%m%d')

# 추가 파생 변수 생성
cost['7day_WaterUsage_Volatility'] = calculate_rolling_std(cost, 'WaterUsage')
cost['7day_FertilizerUsage_Volatility'] = calculate_rolling_std(cost, 'FertilizerUsage')
cost['WaterUsage_Recent_Trend'] = calculate_recent_trend(cost, 'WaterUsage')
cost['FertilizerUsage_Recent_Trend'] = calculate_recent_trend(cost, 'FertilizerUsage')

# 일부 NaN 값 처리
required_columns = ['7day_avg_WaterUsage', '7day_avg_FertilizerUsage', '7day_avg_CO2Usage', '7day_WaterUsage_Volatility',
                    '7day_FertilizerUsage_Volatility', 'WaterUsage_Recent_Trend', 'FertilizerUsage_Recent_Trend']
cost = cost.dropna(subset=required_columns)

# 예: 물 사용량이 처음 발생한 날부터의 경과일
first_water_usage_date = cost[cost['WaterUsage'] > 0]['date'].iloc[0]
cost['Days_Since_First_WaterUsage'] = (cost['date'] - first_water_usage_date).dt.days

# 물 사용량 대비 CO2 사용량의 비율
cost = calculate_ratio(cost, 'WaterUsage', 'CO2Usage', 'Water_to_CO2_Usage_Ratio')

# 비료 사용량 대비 물 사용량의 비율
cost = calculate_ratio(cost, 'FertilizerUsage', 'WaterUsage', 'Fertilizer_to_Water_Usage_Ratio')

# 요일 정보 추가 (0: 월요일, 6: 일요일)
cost['day_of_week'] = cost['date'].dt.dayofweek

# 하루 평균 Mist 사용 시간
cost['avg_mist_usage_per_day'] = cost['MistUsageTime'] / (cost['date'].diff().dt.days.fillna(0) + 1)

# 결과 확인
print(cost.head())

            date   ph   ec  frtstCo     tcdmt      grwtLt  fcluHg  lefLt  \
18351 2022-09-26  0.0  0.0      0.0  6.973377   23.358008     0.0    0.0   
18423 2022-09-26  0.0  0.0      0.0  7.249897   19.466223     0.0    0.0   
55504 2022-09-26  0.0  0.0      0.0  6.898958  192.370414     0.0    0.0   
3563  2022-09-26  0.0  0.0      0.0  8.454030   17.550623     0.0    0.0   
3994  2022-09-26  0.0  0.0      0.0  8.571427  211.112435     0.0    0.0   

       lefCunt  lefBt  ...  ph_ec_interaction  7day_WaterUsage_Volatility  \
18351      0.0    0.0  ...                0.0                         0.0   
18423      0.0    0.0  ...                0.0                         0.0   
55504      0.0    0.0  ...                0.0                         0.0   
3563       0.0    0.0  ...                0.0                         0.0   
3994       0.0    0.0  ...                0.0                         0.0   

       7day_FertilizerUsage_Volatility  WaterUsage_Recent_Trend  \
18351        

<ipython-input-13-31d39b00406e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name] = np.where(df[denominator] != 0, df[numerator] / df[denominator], 0)
<ipython-input-13-31d39b00406e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name] = np.where(df[denominator] != 0, df[numerator] / df[denominator], 0)
<ipython-input-13-31d39b00406e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [14]:
cost.columns

Index(['date', 'ph', 'ec', 'frtstCo', 'tcdmt', 'grwtLt', 'fcluHg', 'lefLt',
       'lefCunt', 'lefBt', 'frmAr', 'frmDov', 'outtrn', 'outtrn_cumsum',
       'WaterUsage', 'WaterCost', 'FertilizerUsage', 'FertilizerCost',
       'HeatingEnergyUsage', 'HeatingEnergyCost', 'CO2Usage', 'CO2Cost',
       'MistUsageTime', 'Mist Cost', 'HeatingEnergyUsage_cumsum',
       'CO2_Usage_Cost_Ratio', 'Mist_Time_Cost_Ratio', '7day_avg_WaterUsage',
       '7day_avg_FertilizerUsage', '7day_avg_CO2Usage', 'ph_ec_interaction',
       '7day_WaterUsage_Volatility', '7day_FertilizerUsage_Volatility',
       'WaterUsage_Recent_Trend', 'FertilizerUsage_Recent_Trend',
       'Days_Since_First_WaterUsage', 'Water_to_CO2_Usage_Ratio',
       'Fertilizer_to_Water_Usage_Ratio', 'day_of_week',
       'avg_mist_usage_per_day'],
      dtype='object')

# 훈련/테스트 데이터셋 생성

In [15]:
# 모든 컬럼 리스트 생성
all_columns = set(con.columns).union(set(cost.columns))

# 각 데이터 프레임에 없는 컬럼을 0으로 채우기
for column in all_columns:
    if column not in con.columns:
        con[column] = 0
    if column not in cost.columns:
        cost[column] = 0

# 두 데이터 프레임 합치기
combined_df = pd.concat([con, cost], axis=0).sort_values(by='date').reset_index(drop=True)

# 'date' 컬럼을 사용하여 'year', 'month', 'day' 컬럼 생성
combined_df['year'] = combined_df['date'].dt.year
combined_df['month'] = combined_df['date'].dt.month
combined_df['day'] = combined_df['date'].dt.day

# 'date' 컬럼 타입 변경 (필요한 경우에만)
combined_df['date'] = combined_df['date'].astype(int)

print(combined_df[['year', 'month', 'day']].head())

<ipython-input-15-783bac8e3e25>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost[column] = 0
<ipython-input-15-783bac8e3e25>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost[column] = 0
<ipython-input-15-783bac8e3e25>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

   year  month  day
0  2017      7    9
1  2017      7    9
2  2017      7    9
3  2017      7    9
4  2017      7    9


In [16]:
# from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# X = combined_df[combined_df.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum', 'date', 'year']).columns]
# Y = combined_df[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
# X_train, X_test, y_train, y_test = train_test_split(
#     X, Y, test_size=0.2, random_state=42
# )

In [17]:
# X.columns #파생변수 O

In [18]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X = df[df.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = df[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [19]:
X.columns # 파생변수 X

Index(['date', 'inTp', 'inHd', 'otmsuplyqy', 'acSlrdQy', 'cunt', 'ph', 'outTp',
       'outWs', 'inCo2', 'ec', 'frmYear', 'frmWeek', 'frtstGrupp', 'flanGrupp',
       'frtstCo', 'tcdmt', 'frmhsFclu', 'hvstGrupp', 'grwtLt', 'fcluHg',
       'lefLt', 'hvstCo', 'lefCunt', 'lefBt', 'stemThck', 'frmAr', 'frmDov',
       'outtrn', 'WaterUsage', 'WaterCost', 'FertilizerUsage',
       'FertilizerCost', 'HeatingEnergyUsage', 'HeatingEnergyCost', 'CO2Usage',
       'CO2Cost', 'MistUsageTime', 'Mist Cost'],
      dtype='object')

# 모델 정의 및 학습, 예측

In [20]:
# from sklearn.ensemble import RandomForestRegressor

# # Initialize and train the LinearRegression model
# model = RandomForestRegressor()
# model.fit(X_train, y_train)

# # Predict 'y' values using the trained model
# y_pred = model.predict(X_test)

In [21]:
from sklearn.linear_model import LinearRegression, LassoCV, Ridge, LassoLarsCV, ElasticNetCV, MultiTaskLassoCV
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # 이 줄은 아래 HistGradientBoostingRegressor를 사용하기 전에 필요합니다.
from sklearn.ensemble import HistGradientBoostingRegressor

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [22]:
class SimpleEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, models):
        self.models = [MultiOutputRegressor(model) for model in models]

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
        return self

    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        return np.mean(predictions, axis=0)

#linearRegressor = LinearRegression()
#lassocv = LassoCV()
#ridge = Ridge()
#lassolarscv = LassoLarsCV()
#elasticnetcv = ElasticNetCV()
model_xgb = xgb.XGBRegressor()
GBoost = GradientBoostingRegressor()
LightGB = lgb.LGBMRegressor(force_row_wise='true')
RF = RandomForestRegressor()
ETR = ExtraTreesRegressor()
HGBR = HistGradientBoostingRegressor()

# 테스트 세트에 대한 예측
models = [GBoost, LightGB, RF, model_xgb, ETR, HGBR]
model = SimpleEnsemble(models)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 9000
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 39
[LightGBM] [Info] Start training from score 25537.923871
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 9000
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 39
[LightGBM] [Info] Start training from score 144306.559980


# 성능 평가

In [23]:
# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_test, y_pred)


rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

# ------------------------------------------------
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 26822.116899029188
R2_score: 0.9974610127694015
